In [ ]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

In [ ]:
# Gerando Dados Sintéticos 

# Quantidade de pontos em cada classe
nA = 50
nB = 60
nC = 55

# Posição dos centróides
centroidA = np.array([7, 8])
centroidB = np.array([3, 2])
centroidC = np.array([8, 3.5])
centsDF = pd.DataFrame({'x': [centroidA[0], centroidB[0], centroidC[0]], 
                        'y': [centroidA[1], centroidB[1], centroidC[1]],
                        'classe':['centroid']*3})

# Variâcia de cada classe
varA = .9
varB = 1.1
varC = 1.

# Gerando coordenadas x para cada classe (X = média + valor_aleatório_distribuição_normal * variância)
Ax = centroidA[0] + np.random.randn(nA) * varA
Bx = centroidB[0] + np.random.randn(nB) * varB
Cx = centroidC[0] + np.random.randn(nC) * varC

# Gerando coordenadas y para cada classe
Ay = centroidA[1] + np.random.randn(nA) * varA
By = centroidB[1] + np.random.randn(nB) * varB
Cy = centroidC[1] + np.random.randn(nC) * varC

# Concatenando todos os x e y em vetores únicos 
dadosX = np.concatenate([Ax, Bx, Cx])
dadosY = np.concatenate([Ay, By, Cy])

# Combinando os dados x e y em um único vetor com dimensões -> (nA+nB+nC, 2)
dados = np.stack([dadosX, dadosY]).T

# Criando o vetor com os rótulos de cada ponto
classes = ['A'] * nA + ['B'] * nB + ['C'] * nC

# Criando DataFrame do Pandas com os dados gerados acima
DF = pd.DataFrame({'x': dadosX, 'y': dadosY, 'classe': classes})

# Paleta de cores para os gráficos -> (Ciano, Azul, Verde, Vermelho)
palette = ['#00ffff', '#0000ff', '#00ff00', '#ff0000']

sns.scatterplot(data=pd.concat([DF, centsDF]),
                x='x', 
                y='y', 
                hue='classe', 
                palette=palette)


In [ ]:
# Utilizado o KMeans do SK Learn
kmeans = KMeans(n_clusters=3)
kmeans.fit(dados)

DF['sklearn'] = kmeans.labels_
Cs = kmeans.cluster_centers_
sklearnDF = pd.DataFrame({'x': Cs[:,0], 
                        'y': Cs[:,1],
                        'sklearn':['centroid']*3})

plt.figure(figsize=(18,8))
plt.subplot(1,2,1)
sns.scatterplot(data=pd.concat([DF, centsDF]), x='x', y='y', hue='classe', palette=palette)
plt.subplot(1,2,2)
sns.scatterplot(data=pd.concat([DF, sklearnDF]), x='x', y='y', hue='sklearn', palette=palette)

In [ ]:
# Função externa para plotar passo a passo do nosso modelo
  # Ela não é necessária, coloquei apenas para facilitar a visualização
def print_steps(dados, labels, centroids):
  plt.figure()
  sns.scatterplot(x=np.concatenate([dados[:,0], centroids[:,0]]), 
                  y=np.concatenate([dados[:,1], centroids[:,1]]), 
                  hue=np.concatenate([labels, [-1]*len(centroids)]),
                  palette=list(reversed(palette))[:len(set(labels))+1])
class MLDLKMeans:
  def __init__(self, n_clusters):
    self.n = n_clusters

  def fit(self, dados, printSteps=False):
    # passo 0 -> Inicialização dos clusters e do modelo
      # Dica: para amostrar aleatoriamente pontos do nosso dataset, podemos usar a função random.sample do python
      # por exemplo: centroids = random.sample(list(dados), n_clusters)
    Cs = np.array(random.sample(list(dados), self.n))
    self.labels_ = np.zeros(dados.shape[0]).astype(int)

    # Loop
    while True:
      # Função de debug para plotar cada iteração (apenas para ajudar, não é necessária)
      if printSteps and dados.shape[1] == 2:
        print_steps(dados, self.labels_, Cs)

      # passo 1 -> Atribuir rótulos aos dados conforme centróide mais próximo
        # Para isso, precisamos percorrer todos os dados,
        # comparar a distância com cada um dos centróides e 
        # escolher o index da menor distância euclidiana
        for i, point in enumerate(dados):
          self.labels_[i] = np.argmin([np.linalg.norm(point - centroid) for centroid in Cs])

      # passo 2 -> Recalcular centroides a partir dos novos rótulos gerados no passo 1
        # Para isso, iremos computar a média de todos os pontos (dados)
        # pertencentes a cada um dos grupos (relativos aos centróides)
        lastCs = Cs
        Cs = np.array([sum(dados[self.labels_ == i]) / sum(self.labels_ == i) for i in range(self.n)])

      # passo X -> Condição de parada é quando não há mais alteração da posição dos centroides
        # Para isso, precisamos ter armazenado os valores antigos dos centróides
        # (antes da última alteração) e comparar com o atual. Caso não haja diferença,
        # então podemos quebrar o loop e encerrar nossa função fit()
        if np.linalg.norm(lastCs - Cs) <= 1e-8:
          break

    self.cluster_centers_ = Cs
    return self

In [ ]:
# Utilizando o nosso modelo de KMeans para agrupar os mesmos dados
mldl = MLDLKMeans(n_clusters=3)
mldl.fit(dados, printSteps=True)

DF['mldl'] = mldl.labels_

Ms = mldl.cluster_centers_
mldlDF = pd.DataFrame({'x': Ms[:,0], 
                       'y': Ms[:,1],
                       'mldl':['centroid']*3})


In [ ]:
# Plotando todos os resultados finais
plt.figure(figsize=(18,8))
plt.subplot(1,3,1)
sns.scatterplot(data=pd.concat([DF, centsDF]), x='x', y='y', hue='classe', palette=palette)
plt.subplot(1,3,2)
sns.scatterplot(data=pd.concat([DF, sklearnDF]), x='x', y='y', hue='sklearn', palette=palette)
plt.subplot(1,3,3)
sns.scatterplot(data=pd.concat([DF, mldlDF]), x='x', y='y', hue='mldl', palette=palette)